## 학습용 데이터 만들기
 - 1. 침수데이터 결측치 해결
 - 2. 침수데이터와 비침수 데이터 합치기
 - 3. 필요한 데이터만 뽑아내기
 - 4. 가중치 주기
 - 5. 학습용 데이터 저장

In [1]:
import pandas as pd
import numpy as np

In [2]:
BF_R_P = pd.read_csv('../data/processing_data/2nd/busan_flood_geo_rain.csv')

In [3]:
BF_R_P['FLOOD'] = np.nan

In [4]:
SEQ_list = BF_R_P['SEQ'].unique()

In [5]:
for seq in SEQ_list:
    #사건 중 최대 강우량 인덱스를 마지막 인덱스로 지정
    end_idx = BF_R_P.loc[(BF_R_P["SEQ"]==seq)]["HOUR_RAINFALL"].idxmax()
    
    #시작 인덱스를 마지막 인덱스와 같은 값으로 정하고 아래에서 -1씩 뺄것
    start_idx = end_idx
    
    #해당 사건의 최대 "FLUD_DPWT"(침수심) 값을 저장
    FLUD_DPWT_max = BF_R_P.loc[end_idx,"FLUD_DPWT"]
    
    #해당 행의 강우 값을 저장하여 5이하가 나올 때까지 사용할 변수
    time_rain = BF_R_P.loc[(BF_R_P["SEQ"]==seq)]["HOUR_RAINFALL"].max()

    #해당 행의 강우가 5이하이면 스톱하여 그 떄의 인덱스를 찾아 start_idx로 지정
    while time_rain >=5:
        start_idx -= 1
        time_rain = BF_R_P.loc[start_idx,"HOUR_RAINFALL"]
    
    # 최대침수심에서 0인 곳까지 인덱스의 범위
    n=end_idx-start_idx
    
    # 침수심 결측치를 채울 첫 행은 강우가 5mm이하 이므로 0
    BF_R_P.loc[start_idx,"FLUD_DPWT"]=0
    
    # "FLOOD"(침수여부)에 대해서 시작 인덱스와 마지막 인덱스는 정해져있음
    # 시작 인덱스의 "FLOOD" == 0
    # 마지막 인덱스의 "FLOOD" == 1
    BF_R_P.loc[start_idx,"FLOOD"] = 0
    BF_R_P.loc[end_idx,"FLOOD"] = 1
    
    # 시작 인덱스와 마지막 인덱스 사이의 침수심 결측치를 선형적인 값을 채움
    for i in range(1,n):
        FLUD_DPWT = (FLUD_DPWT_max/(n))*i
        BF_R_P.loc[start_idx+i,"FLUD_DPWT"]=FLUD_DPWT
        if FLUD_DPWT >= 0.2 :
            BF_R_P.loc[start_idx+i,"FLOOD"]=1
        else:
            BF_R_P.loc[start_idx+i,"FLOOD"]=0


In [6]:
BF_R_P

,SEQ,DISTRICT,ADM_CD,FLUD_TIME,F_BEGIN_DE,F_END_DE,F_BEGIN_TM,F_END_TM,ZONE,DONG,FLUD_DPWT,SLOPE_L,SLOPE_H,SLOPE_AVG,HIGH,HOUR_RAINFALL,PRECI_YYMMDDHH,PRECI_HH,FLOOD
0,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,0.0,0.0,2,1.0,1.98,1.5,20090716,1,NaN
1,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,0.0,0.0,2,1.0,1.98,5.0,20090716,2,NaN
2,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,0.0,0.0,2,1.0,1.98,3.0,20090716,3,NaN
3,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,0.0,0.0,2,1.0,1.98,3.5,20090716,4,NaN
4,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,0.0,0.0,2,1.0,1.98,3.5,20090716,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,102001,강동1지구,2644000000,13.0,20190719,20190720,2300.0,1200.0,강서구,대저2동,0.0,0.0,2,1.0,1.57,0.9,20190720,20,NaN
3956,102001,강동1지구,2644000000,13.0,20190719,20190720,2300.0,1200.0,강서구,대저2동,0.0,0.0,2,1.0,1.57,0.8,20190720,21,NaN
3957,102001,강동1지구,2644000000,13.0,20190719,20190720,2300.0,1200.0,강서구,대저2동,0.0,0.0,2,1.0,1.57,0.5,20190720,22,NaN
3958,102001,강동1지구,2644000000,13.0,20190719,20190720,2300.0,1200.0,강서구,대저2동,0.0,0.0,2,1.0,1.57,1.6,20190720,23,NaN


In [8]:
BUnF = pd.read_csv('../data/processing_data/2nd/busan_unflood_geo_rain.csv')

In [9]:
Flood_Exist = BF_R_P[BF_R_P['FLOOD']>=0]

In [10]:
Unf_match = BUnF.loc[Flood_Exist[Flood_Exist['FLOOD']==1].index]

In [11]:
df = pd.concat([Flood_Exist,Unf_match])

In [13]:
PIH_merge = pd.read_csv('../Data/processing_data/2nd/PIH_merge.csv')

In [29]:
training_data = pd.merge(df,PIH_merge,on='ZONE',how='left')

In [30]:
training_data.head(20)

,SEQ,DISTRICT,ADM_CD,FLUD_TIME,F_BEGIN_DE,F_END_DE,F_BEGIN_TM,F_END_TM,ZONE,DONG,...,PRECI_YYMMDDHH,PRECI_HH,FLOOD,CODE,IMP_SUR_RATIO,ZONE_AREA,MANHOLES_COUNT,PUMP_COUNT,PUMP_RATIO,MANHOLES_RATIO
0,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,...,20090716,5,0.0,2638000000,38.20,471100000.0,8331,7.0,1.485884e-08,0.017684
1,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,...,20090716,6,0.0,2638000000,38.20,471100000.0,8331,7.0,1.485884e-08,0.017684
2,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,...,20090716,7,1.0,2638000000,38.20,471100000.0,8331,7.0,1.485884e-08,0.017684
3,30581,장림지구,2638010500,4.0,20090716,20090716,800.0,1200.0,사하구,장림동,...,20090716,8,1.0,2638000000,38.20,471100000.0,8331,7.0,1.485884e-08,0.017684
4,30582,남천지구,2650010500,4.0,20090716,20090716,600.0,1000.0,수영구,남천동,...,20090716,5,0.0,2650000000,61.44,103500000.0,1930,0.0,0.000000e+00,0.018647
5,30582,남천지구,2650010500,4.0,20090716,20090716,600.0,1000.0,수영구,남천동,...,20090716,6,1.0,2650000000,61.44,103500000.0,1930,0.0,0.000000e+00,0.018647
6,30582,남천지구,2650010500,4.0,20090716,20090716,600.0,1000.0,수영구,남천동,...,20090716,7,1.0,2650000000,61.44,103500000.0,1930,0.0,0.000000e+00,0.018647
7,30582,남천지구,2650010500,4.0,20090716,20090716,600.0,1000.0,수영구,남천동,...,20090716,8,1.0,2650000000,61.44,103500000.0,1930,0.0,0.000000e+00,0.018647
8,30583,송정지구1-1,2635010800,4.0,20090716,20090716,700.0,1100.0,해운대구,송정동,...,20090716,5,0.0,2635000000,27.63,523200000.0,8794,11.0,2.102446e-08,0.016808
9,30583,송정지구1-1,2635010800,4.0,20090716,20090716,700.0,1100.0,해운대구,송정동,...,20090716,6,0.0,2635000000,27.63,523200000.0,8794,11.0,2.102446e-08,0.016808


In [31]:
training_data.columns

Index(['SEQ', 'DISTRICT', 'ADM_CD', 'FLUD_TIME', 'F_BEGIN_DE', 'F_END_DE',
       'F_BEGIN_TM', 'F_END_TM', 'ZONE', 'DONG', 'FLUD_DPWT', 'SLOPE_L',
       'SLOPE_H', 'SLOPE_AVG', 'HIGH', 'HOUR_RAINFALL', 'PRECI_YYMMDDHH',
       'PRECI_HH', 'FLOOD', 'CODE', 'IMP_SUR_RATIO', 'ZONE_AREA',
       'MANHOLES_COUNT', 'PUMP_COUNT', 'PUMP_RATIO', 'MANHOLES_RATIO'],
      dtype='object')

In [32]:
training_data = training_data[['PUMP_RATIO','HOUR_RAINFALL','IMP_SUR_RATIO','SLOPE_AVG',\
                               'HIGH','FLUD_DPWT','FLOOD']]

In [37]:
training_data.head(20)

,PUMP_RATIO,HOUR_RAINFALL,IMP_SUR_RATIO,SLOPE_AVG,HIGH,FLUD_DPWT,FLOOD,Hourly_Rainfall_Weight
0,1.485884e-08,3.5,38.20,1.00,1.98,0.000000,0.0,0
1,1.485884e-08,8.5,38.20,1.00,1.98,0.144333,0.0,0
2,1.485884e-08,39.0,38.20,1.00,1.98,0.288667,1.0,0
3,1.485884e-08,78.0,38.20,1.00,1.98,0.433000,1.0,0
4,0.000000e+00,3.5,61.44,4.50,3.12,0.000000,0.0,0
5,0.000000e+00,8.5,61.44,4.50,3.12,0.226667,1.0,0
6,0.000000e+00,39.0,61.44,4.50,3.12,0.453333,1.0,0
7,0.000000e+00,78.0,61.44,4.50,3.12,0.680000,1.0,0
8,2.102446e-08,3.5,27.63,1.00,2.10,0.000000,0.0,0
9,2.102446e-08,8.5,27.63,1.00,2.10,0.183333,0.0,0


In [51]:
training_data['Hourly_Rainfall_Weight'] = pd.Series(dtype=float)
training_data['Imprevious_Surface_Weight'] = pd.Series(dtype=float)
training_data['SLOPE_AVG_Weight'] = pd.Series(dtype=float)

In [27]:
#가중치

## 강우량 가중치

In [48]:
for idx,col in enumerate(training_data['HOUR_RAINFALL']):
    if col <= 15:
        training_data['Hourly_Rainfall_Weight'][idx] = col / (46)
    elif col > 15 and col <= 35:
        training_data['Hourly_Rainfall_Weight'][idx] = col / (18 * 2**2)
    elif col > 35 and col <= 60:
        training_data['Hourly_Rainfall_Weight'][idx] = col / (17 * 3**2)
    elif col > 60 and col <= 80:
        training_data['Hourly_Rainfall_Weight'][idx] = col / (11 * 4**2)
    elif col > 80:
        training_data['Hourly_Rainfall_Weight'][idx] = col / (6.28 * 5**2)

## 불투수면 가중치

In [ ]:
for idx,col in enumerate(Busanaccident['Impervious_Surface']):
    if col <= 15:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / 6.66
    elif col > 15 and col <= 25:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (20 * 2)
    elif col > 25 and col <= 40:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (13.33 * 3)
    elif col > 40 and col <= 55:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (33.33 * 4)
    elif col > 55:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (26.67 * 5)

## 경사도 가중치

In [ ]:
Busanaccident['SLOPE_AVG_Weight'] = Busanaccident['SLOPE_AVG']
for idx,col in enumerate(Busanaccident['SLOPE_AVG']):
    if col <= 10:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (41.67 / 5**2 )
    elif col > 10 and col <= 15:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (25 / 4**2)
    elif col > 15 and col <= 25:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (16.67 / 3**2)
    elif col > 25 and col <= 50:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (8.33 / 2**2)
    elif col > 50:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (8.33 / 1**2)

## 고도 가중치

In [ ]:
Busanaccident['HIGH_Weight'] = Busanaccident['HIGH']
for idx,col in enumerate(Busanaccident['HIGH']):
    if col <= 20:
        Busanaccident['HIGH_Weight'][idx] = col / (79.62 / 1**2)
    elif col > 20 and col <= 40:
        Busanaccident['HIGH_Weight'][idx] = col / (10.18 / 2**2)
    elif col > 40 and col <= 60:
        Busanaccident['HIGH_Weight'][idx] = col / (4.63 / 3**2)
    elif col > 60 and col <= 80:
        Busanaccident['HIGH_Weight'][idx] = col / (2.78 / 4**2)
    elif col > 80:
        Busanaccident['HIGH_Weight'][idx] = col / (2.78 / 5**2)